# HW5 - Brian Hicks, Joe Everton
## CS 498, Applied Machine Learning
Using the ADL dataset, available [here](https://archive.ics.uci.edu/ml/datasets/Dataset+for+ADL+Recognition+with+Wrist-worn+Accelerometer#).

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.metrics.pairwise import paired_euclidean_distances, euclidean_distances
# import scipy.spatial.distance
# from scipy.linalg import eigh as largest_eigh
# import sklearn.metrics
# from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
import os
import ntpath
# https://stackoverflow.com/questions/8384737/extract-file-name-from-path-no-matter-what-the-os-path-format
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

def load_data():
  dataset = []
  labels = []
  label_index = 0
  for dirname, dirnames, filenames in os.walk('HMP_Dataset'):
      # target all leaf directories not ending with '_MODEL'.
      if len(dirnames) == 0 and '_MODEL' not in dirname:
        class_name = path_leaf(dirname)
        labels.append(class_name)
#         print("dirname: {}, filenames: {}".format(dirname, len(filenames)))
        for filename in filenames:
#             print(os.path.join(dirname, filename))
            dataset.append((np.loadtxt(os.path.join(dirname, filename)), label_index))
        label_index += 1
  return dataset, labels

In [ ]:
dataset, labels = load_data()

In [ ]:
def split_and_flatten_example(example, k, offset):
  chunk_list = np.array_split(example, np.arange(k, len(data), step=k))
  if len(chunk_list[-1]) != k:
    chunk_list = chunk_list[:-1]
  if offset != k:
    chunk_list = chunk_list[1:]
  chunk_list = [np.ravel(chunk) for chunk in chunk_list]
  return chunk_list

In [ ]:
class h_kmeans:
  n_jobs = -1
  def __init__(self, k_breadth):
    # k_breadth is a vector of K for each depth.  The depth of the tree == length of k_breadth.
    self.my_k = k_breadth
    # print ("Initializing KMeans, n_clusters = {}".format(self.my_k[0]))
    self.km = KMeans(n_clusters=self.my_k[0], n_jobs=h_kmeans.n_jobs)
    self.children = []
    if len(k_breadth) > 1:
      # Remove my k, pass the rest to the children.
      self.children = [h_kmeans(k_breadth[1:]) for km in range(self.my_k[0])]
      # self.children = [h_kmeans(k_breadth[1:])] * self.my_k[0]

  def fit(self, X):
    self.km.fit(X)
    if len(self.children) > 0:
      clusters = self.km.predict(X)
      for index, child_node in enumerate(self.children):
        # pick out the rows that were predicted for cluster child_node.
        match = clusters == index
        print("fitting child using {} rows.".format(np.sum(match)))
        child_node.fit(X[match])
  
  def predict(self, X):
    if len(self.children) == 0:
      clusters = self.km.predict(X)
      return clusters
    if len(self.children) > 0:
      clusters = self.km.predict(X)
      depth_clusters = []
      # TODO: We could pass sample_weight, and have it be 0 where it doesn't 
      # belong to this child, and 1 where it does.  That would get rid of the
      # for loop.
      for i in range(len(X)):
        # a cluster anywhere on the tree should have a unique id for histogramming
        # This math is supposed to use children results to get the flattened cluster
        # index.
        depth_clusters.append(
          clusters[i] * self.myk[1] + self.children[clusters[i]].predict(X[i]))
      return depth_clusters

In [ ]:
# Cut into k-sized chunks and fill the dictionary.
k = 32
# TODO: dictionary is a poor name for this.
dictionary = []
for example in dataset:
  data = example[0]
  for offset in range(0, k, 11): # 0, 11, 22, to get about 3 x 13544 chunks.
    split_data = split_and_flatten_example(data, k, k - offset)
    # print("example size: {} split into {} chunks".format(data.shape,len(split_data)))
    dictionary.extend(split_data)
dictionary = np.array(dictionary) # convert from list of arrays to 2D array

In [ ]:
len(dictionary)


In [ ]:
# Run k-means
# km = KMeans(n_clusters=480, random_state=0, n_jobs=2).fit(dictionary)
km = h_kmeans((40,12))
km.fit(dictionary)
# km.predict

In [ ]:
km = KMeans(n_clusters=480, random_state=0, n_jobs=2).fit(dictionary)

# Snippets

## Split experiments.
Probably going to use kfold anyway.

## Experiments with NDArray Views.
Never did find a way to use integer indexing to produce a view.

In [ ]:
def set_all_to_value(a, value):
  a.fill(value)

foo = np.tile(np.array(np.arange(5)), 5)
set_all_to_value(foo.view()[[1, 2, 8, 11]], -1)
foo[[1, 2, 8, 11]][2] = -1
foo

In [ ]:
# x = np.array([(1, 2),(3,4)], dtype=[('aasdf', np.int8), ('bwerewr', np.int8)])
x = np.array([1, 2, 3, 4])# , dtype=[('aasdf', np.int8), ('bwerewr', np.int8)])
print(x)
#xv = x.view(dtype=np.int8).reshape(-1,2)
xv = x.view()
print(xv)
y = xv[[1, 2],]
y[1] = -20
print(y)
print(xv)
print(x)

In [ ]:
x = np.arange(5)
x

y = x[::2]
y

y[0] = 3
x

np.sum(x == 3)
# x == 3